In [6]:
import pandas as pd

In [159]:
#PARTICIPANT FRAMES

df = pd.read_json('data/IRON-Timeline/LA2_954143469_timeline.json')

df_loop = pd.DataFrame(df["info"]["frames"][15]["participantFrames"])

df_loop = df_loop.transpose().drop(columns=["championStats","currentGold","goldPerSecond","participantId","position","timeEnemySpentControlled","level"])

df_loop.reset_index(drop=True, inplace=True)

damage_stats = df_loop["damageStats"]

df_expanded = pd.json_normalize(damage_stats)[["totalDamageDone","totalDamageDoneToChampions","totalDamageTaken","trueDamageDone","trueDamageDoneToChampions","trueDamageTaken"]]

pd_all = pd.concat([df_loop, df_expanded], axis=1).drop(columns=["damageStats"])

pd_all["playerId"] = pd_all.index

pd_all


,jungleMinionsKilled,minionsKilled,totalGold,xp,totalDamageDone,totalDamageDoneToChampions,totalDamageTaken,trueDamageDone,trueDamageDoneToChampions,trueDamageTaken,playerId
0,0,90,5626,7272,53310,9209,7356,10657,2259,0,0
1,56,5,4267,3854,41066,3205,10440,6351,268,260,1
2,0,44,3067,4536,25216,3363,8699,2667,190,156,2
3,0,41,3074,3514,21068,2611,6695,0,0,131,3
4,0,4,2397,5236,6581,2487,8173,345,0,496,4
5,0,66,3926,6255,38829,3131,12713,3236,0,2207,5
6,60,23,4832,5695,51838,1825,6751,5842,163,135,6
7,4,85,6509,7856,48753,6828,5263,464,36,278,7
8,0,99,6454,6115,53136,9231,7529,2371,515,96,8
9,0,14,5793,5210,15514,5154,1768,328,328,0,9


In [19]:
#EVENTS & TARGET

import numpy as np
import pandas as pd
import os


look_events=["CHAMPION_SPECIAL_KILL","CHAMPION_KILL","ELITE_MONSTER_KILL","BUILDING_KILL"]

folder_path = "data/IRON-Timeline" #Change to your folder path


def find_indices(data):
    monster_indices = []
    for index, item in enumerate(data):
        if item.get('type') in look_events:
            monster_indices.append(index)
    return monster_indices

def get_events(json_file):
    elements_filtered = []
    for i in range(16):
        try:
            events = json_file["info"]["frames"][i]["events"]
        except IndexError:
            break
        monster_indices = find_indices(events)
        if monster_indices:
            new_elements = [events[e] for e in monster_indices]
            elements_filtered.extend(new_elements)

    events_filtered = pd.DataFrame(elements_filtered)
    return events_filtered


def transform_events(json_file):
    required_columns = ["killerId", "type", "killType", "monsterSubType", "monsterType", "towerType"]

    events = get_events(json_file)

    missing_columns = [col for col in required_columns if col not in events.columns]

    for column in missing_columns:
        events[column] = np.nan

    events = events[required_columns]
    events = events[events["killerId"] != 0]

    events.loc[events["monsterType"] == "DRAGON", "monsterType"] = events.loc[events["monsterType"] == "DRAGON", "monsterSubType"]

    events = events.drop(columns=["monsterSubType"])

    events["kills"] = (events["type"] == "CHAMPION_KILL").astype(int)

    one_hot_encoded = pd.get_dummies(events[['killType', "monsterType", "towerType"]])

    events_encoded = pd.concat([events[['killerId', "kills"]], one_hot_encoded], axis=1)
    
    if 'killType' in events_encoded.columns:
        events_encoded.drop(columns=['killType'], inplace=True)
    if 'monsterType' in events_encoded.columns:
        events_encoded.drop(columns=['monsterType'], inplace=True)
    if 'towerType' in events_encoded.columns:
        events_encoded.drop(columns=['towerType'], inplace=True)
    
    events_encoded = events_encoded.groupby("killerId").sum().reset_index()

    events_encoded["killerId"] = events_encoded["killerId"] - 1

    events_encoded = events_encoded.groupby("killerId").sum().reset_index()

    conditions = [
        (events_encoded['killerId'] >= 0) & (events_encoded['killerId'] <= 4),
        (events_encoded['killerId'] >= 5) & (events_encoded['killerId'] <= 9)
    ]
    values = [100, 200]

    events_encoded['team'] = np.select(conditions, values, default=0)

    events_encoded = events_encoded.groupby("team").sum().reset_index().drop(columns=["killerId"])

    events_encoded["matchId"] = json_file["metadata"]["matchId"]
    
    last_event = json_file["info"]["frames"][-1]["events"][-1]
    events_encoded["target"] = events_encoded["team"].apply(lambda x: 1 if x == last_event.get("winningTeam") else 0)
    
    events_encoded = events_encoded.drop(columns=["team"])

    return events_encoded


all_events = None

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        
        json_file = pd.read_json(file_path)
                        
        if all_events is None:
            all_events = transform_events(json_file)
        else:
            all_events = pd.concat([all_events, transform_events(json_file)], ignore_index=True)
     
columns_to_convert = all_events.columns[~all_events.columns.isin(['matchId',"target"])]   
    
all_events[columns_to_convert] = all_events[columns_to_convert].fillna(0).astype(int) 

all_events

,kills,killType_KILL_FIRST_BLOOD,killType_KILL_MULTI,monsterType_CHEMTECH_DRAGON,monsterType_FIRE_DRAGON,monsterType_RIFTHERALD,towerType_OUTER_TURRET,matchId,target,monsterType_WATER_DRAGON,monsterType_HEXTECH_DRAGON,monsterType_AIR_DRAGON,monsterType_EARTH_DRAGON,towerType_INNER_TURRET,killType_KILL_ACE,towerType_BASE_TURRET,towerType_NEXUS_TURRET
0,7,0,1,0,1,1,0,LA2_1302593691,0,0,0,0,0,0,0,0,0
1,21,1,1,1,0,0,1,LA2_1302593691,1,0,0,0,0,0,0,0,0
2,6,1,0,0,0,0,0,LA2_1216779473,0,1,0,0,0,0,0,0,0
3,10,0,0,0,0,0,1,LA2_1216779473,1,0,0,0,0,0,0,0,0
4,5,1,0,0,0,0,0,LA2_1284938323,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595,15,0,0,0,1,1,0,LA2_1253741053,1,0,0,1,0,0,0,0,0
9596,10,0,1,0,0,0,0,LA2_1296051042,0,0,0,0,0,0,0,0,0
9597,19,1,2,0,1,1,2,LA2_1296051042,1,1,0,0,0,0,0,0,0
9598,10,0,0,0,0,1,0,LA2_1299916741,1,1,0,0,1,0,0,0,0
